<a href="https://colab.research.google.com/github/aulnp/AI-WIDYAMSIB/blob/main/VEHICLE_COUNTER_WIDYAMSIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Install YOLOv5
!pip install -q git+https://github.com/ultralytics/yolov5.git

  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [11]:
import cv2
import numpy as np
import torch
from google.colab import files

In [12]:
# Function to upload video file
uploaded = files.upload()
video_path = next(iter(uploaded))

Saving toll_gate.mp4 to toll_gate (3).mp4


In [13]:
# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
model.eval()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-8-15 Python-3.10.12 torch-2.3.1+cu121 CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 48, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
  

In [8]:
# Open the video file
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object to save output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video_path = '/content/output_video.avi'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Define the counting line (adjusted for left-to-right diagonal)
line_start = (100, 50)  # Kiri atas
line_end = (width - 100, height - 50)  # Kanan bawah

vehicle_count = 0
detected_ids = set()

def is_crossing_line(box, line_start, line_end):
    """
    Check if the center of the bounding box crosses the defined diagonal line.
    Uses line equation and bounding box center.
    """
    x1, y1, x2, y2 = box
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2

    # Equation of line: y = mx + c (slope-intercept form)
    m = (line_end[1] - line_start[1]) / (line_end[0] - line_start[0])
    c = line_start[1] - m * line_start[0]

    # Calculate the y position on the line at center_x
    line_y_at_center_x = m * center_x + c

    # Check if the center of the box crosses the line
    if center_y > line_y_at_center_x - 5 and center_y < line_y_at_center_x + 5:  # Small tolerance for accuracy
        return True
    return False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()

    # Process detections
    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        if conf > 0.5 and (int(cls) == 2 or int(cls) == 5):  # Car or Bus
            box = (int(x1), int(y1), int(x2), int(y2))

            # Draw bounding box and label
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            label = model.names[int(cls)]
            cv2.putText(frame, f'{label}', (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Check if the vehicle crosses the diagonal counting line
            if is_crossing_line(box, line_start, line_end):
                vehicle_count += 1

    # Draw the diagonal counting line (left to right, top to bottom)
    cv2.line(frame, line_start, line_end, (0, 0, 255), 2)
    cv2.putText(frame, f'Vehicles Counted: {vehicle_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write frame to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

In [9]:
# Download the result video
files.download(output_video_path)
print(f"Video hasil telah diunduh: {output_video_path}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video hasil telah diunduh: /content/output_video.avi
